In [1]:
!git clone --recursive https://github.com/Microsoft/LightGBM
%cd /content/LightGBM
!mkdir build
!cmake -DUSE_GPU=1 #avoid ..
!make -j$(nproc)
!sudo apt-get -y install python-pip
!sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
%cd /content/LightGBM/python-package
!sudo python setup.py install --precompile
%cd /content

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment3/'
FOLDERNAME = "Colab Notebooks/mango_class/"
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# this downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd drive/My\ Drive/$FOLDERNAME/

Cloning into 'LightGBM'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 18982 (delta 24), reused 17 (delta 7), pack-reused 18933
Receiving objects: 100% (18982/18982), 12.69 MiB | 28.62 MiB/s, done.
Resolving deltas: 100% (13860/13860), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'compute'
Cloning into '/content/LightGBM/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728        
Receiving objects: 100% (21728/21728), 8.50 MiB | 29.83 MiB/s, done.
Resolving deltas: 100% (17565/17565), done.
Submodule path 'compute': checked out '36c89134d4013b2e5e45bc55656a18bd6141995a'
/content/LightGBM
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compile

In [2]:

# !rm -rf LightGBM
import lightgbm as lgb
# %cd ..
# %cd build
# !cmake -DUSE_GPU=1 ..

In [3]:
import os
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.image as mpimg
from PIL import Image#https://yungyuc.github.io/oldtech/python/python_imaging.html
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import glob
import numpy as np
from PIL import Image#https://yungyuc.github.io/oldtech/python/python_imaging.html


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:

import torchvision.models as models
import torch.nn as nn
import time

import copy
from sklearn.model_selection import train_test_split


In [5]:
import torch
USE_GPU = True

# dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)


using device: cuda


In [6]:
X_train1 = torch.load("dataset/metaX_train1_stacking2")
y_train1 = torch.load("dataset/metay_train1_stacking2")
X_train2 = torch.load("dataset/metaX_train2_stacking2")
y_train2 = torch.load("dataset/metay_train2_stacking2")

X_test1 = torch.load("dataset/metaX_test1_stacking2")
y_test1 = torch.load("dataset/metay_test1_stacking2")
X_test2 = torch.load("dataset/metaX_test2_stacking2")
y_test2 = torch.load("dataset/metay_test2_stacking2")

In [7]:
print(X_train1.shape)
print(y_train1.size())
print(X_train2.shape)
print(y_train2.size())

print(X_test1.shape)
print(y_test1.size())
print(X_test2.shape)
print(y_test2.size())

(11200, 3)
torch.Size([11200])
(11200, 3)
torch.Size([11200])
(800, 3)
torch.Size([800])
(800, 3)
torch.Size([800])


In [8]:
X_train = np.vstack((X_train1, X_train2))#22400,3
y_train = torch.cat((y_train1, y_train2), 0)#22400

# X_test = np.vstack([X_test1, X_test2])
# y_test = np.vstack([y_test1, y_test2])
X_test = (X_test1 + X_test2)/2
print(y_test1[0:12])
print(y_test2[0:12])#must be the same
y_test = y_test1

tensor([2., 0., 1., 0., 2., 2., 1., 2., 1., 2., 2., 0.])
tensor([2., 0., 1., 0., 2., 2., 1., 2., 1., 2., 2., 0.])


In [9]:
print(X_train.shape)

(22400, 3)


In [10]:
from sklearn.preprocessing import StandardScaler

# X_train = X_train.cpu()
y_train = y_train.cpu()
# X_test = X_test.cpu()
y_test = y_test.cpu()

scaler1 = StandardScaler()
scaler1.fit(X_train)
X_train = scaler1.transform(X_train)

# scalar2 = StandardScaler()
# scalar2.fit(X_test)
# X_test = scalar2.transform(X_test)


In [11]:
# X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size = 0.2)

In [12]:
parameters = {
              # 'device':'gpu',
            #   'max_depth': [ 7],
            #   'num_leaves':[ 50],

            #   'min_data_in_leaf':[21],
            #   'min_sum_hessian_in_leaf':[0.001],
              
            #   'feature_fraction': [1.0],
            #   'bagging_fraction': [1.0],
              # # # 'bagging_freq': [2, 4, 5, 6, 8],
            #   'lambda_l1': [0.001],
            #   'lambda_l2': [8],

              'learning_rate': [0.1],
              'n_estimators' : [ 90],
            #   'cat_smooth': [30, 40, 50,60]
}

best_lgbm = None
best_acc = 0
f1, f2 = np.meshgrid(parameters['n_estimators'],parameters['learning_rate'])
f1 = f1.flat[:]
f2 = f2.flat[:]

for i, j in zip(f1, f2):
  lgbm = lgb.LGBMClassifier(
                          boosting_type='gbdt',
                          objective = 'muilti_class',
                          metric = 'multi_logloss',
                          max_depth = 7,
                          num_leaves = 50,
                          learning_rate = 0.1,
                          feature_fraction = 1,
                          min_child_samples=21,
                          min_child_weight=0.001,
                          bagging_fraction = 1,
                          bagging_freq = 2,
                          reg_alpha = 0.001,
                          reg_lambda = 8,
                          cat_smooth = 30, 
                          device = 'cpu',
                          n_estimators = 90,
                        #   gpu_platform_id = 0, 
                        #   gpu_device_id = 0
                          )
  lgbm.fit(X_train, y_train)
  
  y_test_pred = torch.tensor(lgbm.predict(X_test))
  y_train_pred = torch.tensor(lgbm.predict(X_train))
  acc = sum(y_test_pred == y_test).item()/y_test.size(0)
  acc_train = sum(y_train_pred == y_train).item()/y_train.size(0)
  print(f'{i}, {j}, acc in dev:{acc}, acc in train:{acc_train}')
  if best_acc <= acc:
    best_lgbm = lgbm
    best_acc = acc
  
print(best_lgbm, acc) 

[LightGBM] [Warning] feature_fraction is set=1, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
90, 0.1, acc in dev:0.77625, acc in train:0.7673660714285714
LGBMClassifier(bagging_fraction=1, bagging_freq=2, cat_smooth=30, device='cpu',
               feature_fraction=1, max_depth=7, metric='multi_logloss',
               min_child_samples=21, n_estimators=90, num_leaves=50,
               objective='muilti_class', reg_alpha=0.001, reg_lambda=8) 0.77625


In [15]:
!pip install joblib

In [20]:
import joblib
# save model
joblib.dump(best_lgbm, 'model_params/lgbm.pkl')
# load model
gbm_pickle = joblib.load('model_params/lgbm.pkl')
 

In [22]:
y_test_pred = torch.tensor(gbm_pickle.predict(X_test))
y_train_pred = torch.tensor(gbm_pickle.predict(X_train))
acc = sum(y_test_pred == y_test).item()/y_test.size(0)
acc_train = sum(y_train_pred == y_train).item()/y_train.size(0)
print(f' acc in dev:{acc}, acc in train:{acc_train}')

 acc in dev:0.77625, acc in train:0.7673660714285714


In [13]:
import seaborn as sn
cm = np.zeros([3, 3])
for j, i  in zip(y_pred_test,y_test):
    cm[int(i), int(j.item())] += 1
    
df_cm = pd.DataFrame(cm.astype(np.int32),
                     index = [i for i in ["true_A", "true_B", "true_C"]],
                     columns = [i for i in ["pred_A", "pred_B", "pred_C"]])
plt.figure(figsize = (4, 4))
sn.heatmap(df_cm, annot=True, cmap="BuPu")

NameError: ignored